In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from nltk.tokenize import WordPunctTokenizer
import itertools
from pandas_profiling import ProfileReport
import numpy as np


In [2]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_train


,text,source,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK
0,[QUOTE=jessepinkman16;5a50ac34d89b093f368b456e...,kaskus,0,0,0,1
1,"@verosvante kita2 aja nitizen yang pada kepo,t...",instagram,0,0,0,0
2,"""#SidangAhok smg sipenista agama n ateknya mat...",twitter,0,1,1,1
3,@bolususulembang.jkt barusan baca undang2 ini....,instagram,0,0,0,0
4,bikin anak mulu lu nof \nkaga mikir apa kasian...,kaskus,0,0,0,0
...,...,...,...,...,...,...
6990,Mas. Sebenarnya aku gamau bully mas tp kek nya...,instagram,0,0,0,0
6991,"Kdang ngaku nicky minaj, kdang beyonce, kdang ...",instagram,0,0,0,0
6992,"Time kemas sesuatu tadi, terfikir jugak botol ...",twitter,0,0,1,0
6993,"@BasherRL heh kontol, jan sok pemes lu di rp, ...",twitter,1,0,0,1


In [3]:
len(df_train)-len(df_train.drop_duplicates(subset=['text']))


624

In [4]:
df_train = df_train.drop_duplicates(ignore_index=True, subset=['text'])
df_train


,text,source,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK
0,[QUOTE=jessepinkman16;5a50ac34d89b093f368b456e...,kaskus,0,0,0,1
1,"@verosvante kita2 aja nitizen yang pada kepo,t...",instagram,0,0,0,0
2,"""#SidangAhok smg sipenista agama n ateknya mat...",twitter,0,1,1,1
3,@bolususulembang.jkt barusan baca undang2 ini....,instagram,0,0,0,0
4,bikin anak mulu lu nof \nkaga mikir apa kasian...,kaskus,0,0,0,0
...,...,...,...,...,...,...
6366,Iya miris banget. Smua gara2 pengaruh barat ni...,kaskus,0,0,0,0
6367,"Kk luna, harus kuat ya! Harus sabar ngadepin s...",instagram,0,0,0,0
6368,Mas. Sebenarnya aku gamau bully mas tp kek nya...,instagram,0,0,0,0
6369,"Kdang ngaku nicky minaj, kdang beyonce, kdang ...",instagram,0,0,0,0


In [5]:
df_train['text']


0       [QUOTE=jessepinkman16;5a50ac34d89b093f368b456e...
1       @verosvante kita2 aja nitizen yang pada kepo,t...
2       "#SidangAhok smg sipenista agama n ateknya mat...
3       @bolususulembang.jkt barusan baca undang2 ini....
4       bikin anak mulu lu nof \nkaga mikir apa kasian...
                              ...                        
6366    Iya miris banget. Smua gara2 pengaruh barat ni...
6367    Kk luna, harus kuat ya! Harus sabar ngadepin s...
6368    Mas. Sebenarnya aku gamau bully mas tp kek nya...
6369    Kdang ngaku nicky minaj, kdang beyonce, kdang ...
6370    Time kemas sesuatu tadi, terfikir jugak botol ...
Name: text, Length: 6371, dtype: object

In [6]:
print(f'ukuran train : {df_train.shape}')
print(f'ukuran test : {df_test.shape}')


ukuran train : (6371, 6)
ukuran test : (778, 6)


In [7]:
df_train.columns


Index(['text', 'source', 'PORNOGRAFI', 'SARA', 'RADIKALISME',
       'PENCEMARAN_NAMA_BAIK'],
      dtype='object')

In [8]:
df_train['SARA'].value_counts().sum()


6371

In [9]:
np.sum(df_train['SARA'])


1057

In [10]:
df_train[df_train['SARA'] == 1].value_counts().sum()


1057

In [11]:
labels = ['SARA', 'RADIKALISME', 'PENCEMARAN_NAMA_BAIK', 'PORNOGRAFI']


In [12]:
for i in labels:
    print(
        f'presentase data train untuk label {i} adalah {np.sum(df_train[i]) / df_train.value_counts().sum():,.2f}%')


presentase data train untuk label SARA adalah 0.17%
presentase data train untuk label RADIKALISME adalah 0.17%
presentase data train untuk label PENCEMARAN_NAMA_BAIK adalah 0.31%
presentase data train untuk label PORNOGRAFI adalah 0.20%


In [13]:
for j in labels:
    print(
        f'presentase data test untuk label {j} adalah {np.sum(df_test[j]) / df_test.value_counts().sum():,.2f}%')


presentase data test untuk label SARA adalah 0.15%
presentase data test untuk label RADIKALISME adalah 0.16%
presentase data test untuk label PENCEMARAN_NAMA_BAIK adalah 0.31%
presentase data test untuk label PORNOGRAFI adalah 0.22%


# Preprocessing Data

In [31]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, ArrayDictionary, StopWordRemover
list_hapus = open("list_hapus.txt", "r").read().splitlines()


In [15]:
def hapus_baris(text):
    return re.sub('\n', ' ',text)

kata = 'buat apa \ntuh\n'
print(hapus_baris(kata))


buat apa  tuh 


In [16]:
def hapus_backslash(text):
    a = re.sub(r'\\+', '/', text)
    a = re.sub('/n', '\n', a)
    return a


hapus_backslash('bikin anak mulu lu nof \\nkaga mikir apa kasian anak lu nanti bakalan malu punya bapak kelakuan kaya elu\r\n')


'bikin anak mulu lu nof \nkaga mikir apa kasian anak lu nanti bakalan malu punya bapak kelakuan kaya elu\r\n'

In [17]:
a = hapus_backslash(
    'bikin anak mulu lu nof \\nkaga mikir apa kasian anak lu nanti bakalan malu punya bapak kelakuan kaya elu\r\n')
a = hapus_baris(a)
a

'bikin anak mulu lu nof  kaga mikir apa kasian anak lu nanti bakalan malu punya bapak kelakuan kaya elu\r '

In [18]:
def hapus_quote(text):
    text = re.sub('\[', ' [', text)
    text = re.sub('\]', '] ', text)
    text = re.sub('\[quote[^ ]*\].*?\[\/quote\]', ' ', text)
    text = re.sub('\[[^ ]*\]', ' ', text)
    text = re.sub('&quot;', ' ', text)
    return text


text = '[QUOTE=jessepinkman16;5a50ac34d89b093f368b456e]yoiii cuy halo halo bandung[/QUOTE]'

print('Before :', text)
print('After  :', hapus_quote(text))


Before : [QUOTE=jessepinkman16;5a50ac34d89b093f368b456e]yoiii cuy halo halo bandung[/QUOTE]
After  :    yoiii cuy halo halo bandung   


In [19]:
def hapus_link(text):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)


text = 'kemaren kacao bet de www.instagram.com/dj213987qekjhqew'

print('Before :', text)
print('After  :', hapus_link(text))


Before : kemaren kacao bet de www.instagram.com/dj213987qekjhqew
After  : kemaren kacao bet de 


In [20]:
def hapus_spasi_banyak(text):
    return re.sub('  +', ' ', text)


text = 'budi      pergi ke           pasar'

print('Before :', text)
print('After  :', hapus_spasi_banyak(text))


Before : budi      pergi ke           pasar
After  : budi pergi ke pasar


In [21]:
def tokenize_text(text, punct=False):
    text = WordPunctTokenizer().tokenize(text)
    text = [word for word in text if punct or word.isalnum()]
    text = ' '.join(text)
    text = text.strip()
    return text


text = 'kemarin,aku pergi ke dagas.terus ketemu sama Ilham.'

print('Before :', text)
print('After  :', tokenize_text(text))


Before : kemarin,aku pergi ke dagas.terus ketemu sama Ilham.
After  : kemarin aku pergi ke dagas terus ketemu sama Ilham


In [22]:
slang = pd.read_csv('data/slangword.csv')
slang


,original,translated
0,7an,tujuan
1,@,di
2,ababil,abg labil
3,abis,habis
4,acc,accord
...,...,...
2873,tauge,taoge
2874,toge,taoge
2875,taubat,tobat
2876,trilyun,triliun


In [23]:
slang_dict = dict(zip(slang['original'], slang['translated']))


In [24]:
def transform_slang_words(text):
    word_list = text.split()
    word_list_len = len(word_list)
    transformed_word_list = []
    i = 0
    while i < word_list_len:
        if (i + 1) < word_list_len:
            two_words = ' '.join(word_list[i:i+2])
            if two_words in slang_dict:
                transformed_word_list.append(slang_dict[two_words])
                i += 2
                continue
        transformed_word_list.append(
            slang_dict.get(word_list[i], word_list[i]))
        i += 1
    return ' '.join(transformed_word_list)


text = 'dikit lagi abis nih'

print('Before :', text)
print('After  :', transform_slang_words(text))


Before : dikit lagi abis nih
After  : dikit lagi habis ini


In [25]:
def remove_non_alphabet(text):
    output = re.sub('[^a-zA-Z ]+', '', text)
    return output


text = 'kemaren tu123 ada kelinci di kebun.'

print('Before :', text)
print('After  :', remove_non_alphabet(text))


Before : kemaren tu123 ada kelinci di kebun.
After  : kemaren tu ada kelinci di kebun


In [26]:
def remove_twitter_ig_formatting(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'\brt\b', '', text)
    return text


text = '@jonijon menurut saya hal tersebut masih kurang baik dilakukan sih kak'

print('Before :', text)
print('After  :', remove_twitter_ig_formatting(text))


Before : @jonijon menurut saya hal tersebut masih kurang baik dilakukan sih kak
After  :  menurut saya hal tersebut masih kurang baik dilakukan sih kak


In [27]:
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()
print(stopwords)


['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [32]:
removeFactory = StopWordRemoverFactory()
stopwords = removeFactory.create_stop_word_remover()
stopwords_plus = removeFactory.get_stop_words()+list_hapus
dictionary = ArrayDictionary(stopwords_plus)

stopwords = StopWordRemover(dictionary)

def removeStopWords(text):
    stop = stopwords.remove(text)
    return stop


text = 'Andi kerap melakukan transaksi rutin secara daring atau online. Menurut Andi belanja online lebih praktis & murah.'
print('Before :', text)
print('After  :', removeStopWords(text))


Before : Andi kerap melakukan transaksi rutin secara daring atau online. Menurut Andi belanja online lebih praktis & murah.
After  : Andi kerap melakukan transaksi rutin daring online. Menurut Andi belanja online lebih praktis & murah.


In [33]:
stemFactory = StemmerFactory()
stemmer = stemFactory.create_stemmer()


def stemming(text):
    hasil = stemmer.stem(text)
    return hasil


text = "Andi kerap melakukan transaksi rutin secara daring atau online. Menurut Andi belanja online lebih praktis & murah."

print('Before :', text)
print('After  :', stemming(text))


Before : Andi kerap melakukan transaksi rutin secara daring atau online. Menurut Andi belanja online lebih praktis & murah.
After  : andi kerap laku transaksi rutin cara daring atau online turut andi belanja online lebih praktis murah


In [34]:
def remove_repeating_characters(text):
    return ''.join(''.join(s)[:1] for _, s in itertools.groupby(text))


text = 'heyyyyyyyyyyyyyyyyyyyy kenapa tadi?'

print('Before :', text)
print('After  :', remove_repeating_characters(text))


Before : heyyyyyyyyyyyyyyyyyyyy kenapa tadi?
After  : hey kenapa tadi?


In [35]:
def preprocessing(text):
    text = text.lower()
    text = hapus_quote(text)
    text = hapus_backslash(text)
    text = hapus_baris(text)
    # text = hapus_quote(text)
    text = hapus_link(text)
    text = remove_twitter_ig_formatting(text)
    text = tokenize_text(text)
    text = remove_repeating_characters(text)
    # text = remove_non_alphabet(text)
    text = transform_slang_words(text)
    text = remove_non_alphabet(text)
    text = hapus_spasi_banyak(text)
    text = removeStopWords(text)
    # text = stemming(text)
    text = text.lower()
    return text


In [170]:
# print(preprocessing('bikin anak mulu lu nof \\nkaga mikir apa kasian anak lu nanti bakalan malu punya bapak kelakuan kaya elu\r\n'))


buat anak melulu kamu nof mikir apa kasian anak kamu akan malu punya bapak kelakuan kaya kamu


In [36]:
df_train['text'] = df_train['text'].apply(preprocessing)
df_train


C:\Users\ASUS\AppData\Local\Temp\ipykernel_3976\451738679.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['text'] = df_train['text'].apply(preprocessing)


,text,source,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK
0,woi anjing bodoh provinsi paling banyak ngerus...,kaskus,0,0,0,1
1,saja nitizen penasaran keluarga besar sudah pa...,instagram,0,0,0,0
2,sidangahok semoga sipenista agama ateknya mati...,twitter,0,1,1,1
3,jakarta barusan baca undang tetap dibedakan ko...,instagram,0,0,0,0
4,buat anak melulu kamu nof mikir apa kasian ana...,kaskus,0,0,0,0
...,...,...,...,...,...,...
6366,iya miris sekali semua gara pengaruh barat sep...,kaskus,0,0,0,0
6367,luna kuat iya sabar ngadepin sikap pembenci mu...,instagram,0,0,0,0
6368,mas sebenarnya aku mau buly mas seperti nya ma...,instagram,0,0,0,0
6369,kdang mengaku nicky minaj kdang beyonce kdang ...,instagram,0,0,0,0


In [19]:
df_train['text'][6301]


'[QUOTE=Jalan Cinta;5a655723a2c06ebd158b456d]\\n\\n”Menurut pamannya, pria tersebut tidak waras setelah mengalami kecelakaan lalu lintas 15 tahun yang lalu,” kata Nik Ros Azhan, seperti dikutip dari Bernama, Senin (22/1/2018). Polisi, ujar dia, akan terus melacaknya.\\n\\nTsnya ikutan gila :travel\\n\\n[/QUOTE]\\nanjing lu goblog, lebih pilih bela malingsia daripada wanita bangsa sendiiri.\\nketemu gw injek muka lu sekeluarga \\ndasar pengkhianat bangsa:batabig \\n\r\n'

In [38]:
df_train


,text,source,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK
0,woi anjing bodoh provinsi paling banyak ngerus...,kaskus,0,0,0,1
1,saja nitizen penasaran keluarga besar sudah pa...,instagram,0,0,0,0
2,sidangahok semoga sipenista agama ateknya mati...,twitter,0,1,1,1
3,jakarta barusan baca undang tetap dibedakan ko...,instagram,0,0,0,0
4,buat anak melulu kamu nof mikir apa kasian ana...,kaskus,0,0,0,0
...,...,...,...,...,...,...
6366,iya miris sekali semua gara pengaruh barat sep...,kaskus,0,0,0,0
6367,luna kuat iya sabar ngadepin sikap pembenci mu...,instagram,0,0,0,0
6368,mas sebenarnya aku mau buly mas seperti nya ma...,instagram,0,0,0,0
6369,kdang mengaku nicky minaj kdang beyonce kdang ...,instagram,0,0,0,0


In [39]:
df_train.to_csv('data_hasil_baru/kalimat_no_stem_addStopwords.csv', index=False)

In [39]:
df_train['text']


0       woi anjing bodoh provinsi paling banyak ngerus...
1       saja nitizen penasaran keluarga besar sudah pa...
2       sidangahok semoga sipenista agama ateknya mati...
3       jakarta barusan baca undang tetap dibedakan ko...
4       buat anak melulu kamu nof mikir apa kasian ana...
                              ...                        
6366    iya miris sekali semua gara pengaruh barat sep...
6367    luna kuat iya sabar ngadepin sikap pembenci mu...
6368    mas sebenarnya aku mau buly mas seperti nya ma...
6369    kdang mengaku nicky minaj kdang beyonce kdang ...
6370    waktu kemas tadi terfikir jugak botol air keci...
Name: text, Length: 6371, dtype: object

In [30]:
# df_train = df_train


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [41]:
vectorizer = TfidfVectorizer()



In [42]:
result = vectorizer.fit_transform(df_train['text']).toarray()
result


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [43]:
df_baru = pd.DataFrame(result, columns=vectorizer.get_feature_names_out())
df_baru


,ab,abad,abadi,abadinya,abah,abai,abaikan,abang,abangan,abas,...,zona,zoroaster,zouma,zoya,zuhur,zulkarnaen,zulkarnain,zulkifli,zy,zynga
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
df = pd.concat([df_train[['PORNOGRAFI', 'SARA', 'RADIKALISME',
               'PENCEMARAN_NAMA_BAIK']], df_baru], axis=1)
df


,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK,ab,abad,abadi,abadinya,abah,abai,...,zona,zoroaster,zouma,zoya,zuhur,zulkarnaen,zulkarnain,zulkifli,zy,zynga
0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6366,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6367,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6368,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6369,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
df.to_csv('data fix/preprocess_fix_backslash_wo_stemming.csv', index=False)


# Modelling

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV


In [13]:
df = pd.read_csv('data hasil/preprocess_wo_stemming_fix_backslash.csv')



,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK,ab,abad,abadi,abadinya,abah,abai,...,zuhur,zulkarnaen,zulkarnain,zulkifli,zuqfaiugqy,zvjqohw,zwcxnxhd,zwnbzkjn,zy,zynga
0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6990,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6991,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6992,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6993,1,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
x = df.drop(columns=['PORNOGRAFI', 'SARA',
            'RADIKALISME', 'PENCEMARAN_NAMA_BAIK'])
x.head()


,ab,abad,abadi,abadinya,abah,abai,abaikan,abang,abangan,abas,...,zuhur,zulkarnaen,zulkarnain,zulkifli,zuqfaiugqy,zvjqohw,zwcxnxhd,zwnbzkjn,zy,zynga
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# x = df_baru
# x.head()


,aaceab,aacebdbd,ab,abad,abadi,abah,abai,abang,abangan,abas,...,zuhur,zulkarnaen,zulkarnain,zulkifli,zuqfaiugqy,zvjqohw,zwcxnxhd,zwnbzkjn,zy,zynga
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
y = df['PORNOGRAFI']
y.head()


0    0
1    0
2    0
3    0
4    0
Name: PORNOGRAFI, dtype: int64

In [16]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, stratify=y, test_size=0.2, random_state=32)


In [17]:
model = RandomForestClassifier()


In [13]:
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(f'Akurasi : {accuracy_score(y_test, y_pred):,.4f}')


Akurasi : 0.9543


In [14]:
y_pred


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [15]:
print((y_pred == 0).sum())
print((y_pred == 1).sum())


1136
263


In [16]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1084
           1       0.98      0.82      0.89       315

    accuracy                           0.95      1399
   macro avg       0.96      0.91      0.93      1399
weighted avg       0.96      0.95      0.95      1399



In [47]:
Skfold = StratifiedKFold(n_splits=5)
acc = cross_val_score(model, x_train, y_train, cv=Skfold)
print(f'Hasil rata-rata akurasi : {acc.mean()}')
print(f'Rata-rata std : {acc.std()}')
print(
    f'Akurasi agar tidak overfit / underfit {(acc.mean() - acc.std()):,.4f} - {(acc.mean() + acc.std()):,.4f} \n')


Hasil rata-rata akurasi : 0.9512139346355164
Rata-rata std : 0.006240749238835373
Akurasi agar tidak overfit / underfit 0.9450 - 0.9575 



In [17]:
conf = confusion_matrix(y_test, y_pred)
conf


array([[1078,    6],
       [  58,  257]], dtype=int64)

In [18]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 400, 500]
}

rf = RandomForestClassifier()

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2)


In [22]:
grid_search.fit(x_train, y_train)

grid_search.best_params_


Fitting 3 folds for each of 360 candidates, totalling 1080 fits


{'bootstrap': True,
 'max_depth': 110,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 200}

In [23]:
print(grid_search.best_params_)


{'bootstrap': True, 'max_depth': 110, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 200}


In [24]:
best_grid = grid_search.best_estimator_
pred_grid = best_grid.predict(x_test)

print(f'Akurasi : {accuracy_score(y_test, pred_grid):,.4f}')


Akurasi : 0.9421


In [25]:
best_grid


RandomForestClassifier(max_depth=110, min_samples_leaf=3, min_samples_split=8,
                       n_estimators=200)

In [26]:
pred_grid


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [27]:
print((pred_grid == 0).sum())
print((pred_grid == 1).sum())


1159
240


In [28]:
Skfold = StratifiedKFold(n_splits=5)
acc = cross_val_score(best_grid, x_train, y_train, cv=Skfold)
print(f'Hasil rata-rata akurasi : {acc.mean():,.4f}')
print(f'Rata-rata std : {acc.std():,.4f}')
print(
    f'Akurasi agar tidak overfit / underfit {(acc.mean() - acc.std()):,.4f} - {(acc.mean() + acc.std()):,.4f} \n')


Hasil rata-rata akurasi : 0.9366
Rata-rata std : 0.0064
Akurasi agar tidak overfit / underfit 0.9302 - 0.9429 



In [29]:
conf = confusion_matrix(y_test, pred_grid)
conf


array([[1081,    3],
       [  78,  237]], dtype=int64)

In [30]:
print(classification_report(y_test, pred_grid))


              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1084
           1       0.99      0.75      0.85       315

    accuracy                           0.94      1399
   macro avg       0.96      0.87      0.91      1399
weighted avg       0.95      0.94      0.94      1399



In [19]:
from imblearn.over_sampling import SMOTE
from collections import Counter


In [20]:
oversample = SMOTE()
x_smote, y_smote = oversample.fit_resample(x, y)

counter = Counter(y_smote)
print(f'After : {counter}')


After : Counter({0: 5421, 1: 5421})


In [21]:
x_train_smote, x_test_smote, y_train_smote, y_test_smote = train_test_split(
    x_smote, y_smote, test_size=0.2, random_state=42)


In [18]:
rf = RandomForestClassifier()
rf.fit(x_train_smote, y_train_smote)

y_pred_smote = rf.predict(x_test_smote)

acc_tree1 = accuracy_score(y_test_smote, y_pred_smote)

print(
    f'Akurasi : {accuracy_score(y_test_smote, y_pred_smote):,.4f}')


Akurasi : 0.9766


In [19]:
conf = confusion_matrix(y_test_smote, y_pred_smote)
conf


array([[1586,   12],
       [  64, 1591]], dtype=int64)

In [20]:
report = classification_report(y_test_smote, y_pred_smote)
print(report)


              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1598
           1       0.99      0.96      0.98      1655

    accuracy                           0.98      3253
   macro avg       0.98      0.98      0.98      3253
weighted avg       0.98      0.98      0.98      3253



In [76]:
Skfold = StratifiedKFold(n_splits=5)
acc = cross_val_score(rf, x_train_smote, y_train_smote, cv=Skfold)
print(f'Hasil rata-rata akurasi : {acc.mean():,.4f}')
print(f'Rata-rata std : {acc.std():,.4f}')
print(
    f'Akurasi agar tidak overfit / underfit {(acc.mean() - acc.std()):,.4f} - {(acc.mean() + acc.std()):,.4f} \n')


Hasil rata-rata akurasi : 0.9789
Rata-rata std : 0.0065
Akurasi agar tidak overfit / underfit 0.9724 - 0.9854 



In [78]:
grid_search.fit(x_train_smote, y_train_smote)

grid_search.best_params_


Fitting 3 folds for each of 360 candidates, totalling 1080 fits


e:\venv\nlp\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


{'bootstrap': True,
 'max_depth': 110,
 'max_features': 'auto',
 'min_samples_leaf': 3,
 'min_samples_split': 12,
 'n_estimators': 400}

In [79]:
best_grid = grid_search.best_estimator_
pred_grid = best_grid.predict(x_test)

print(f'Akurasi : {accuracy_score(y_test, pred_grid):,.4f}')


Akurasi : 0.9757


In [80]:
conf = confusion_matrix(y_test_smote, y_pred_smote)
conf


array([[1587,   11],
       [  61, 1594]], dtype=int64)

In [81]:
report = classification_report(y_test_smote, y_pred_smote)
print(report)


              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1598
           1       0.99      0.96      0.98      1655

    accuracy                           0.98      3253
   macro avg       0.98      0.98      0.98      3253
weighted avg       0.98      0.98      0.98      3253



In [22]:
def latih(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(f'Akurasi : {accuracy_score(y_test, y_pred):,.4f} \n')
    print('---------- Classification Report ----------')
    print(f'{classification_report(y_test, y_pred)}\n')
    print('---------- Confusion Matrix ----------')
    conf = confusion_matrix(y_test, y_pred)
    print(conf)


In [23]:
def latih_hyperparameter(model, x_train, y_train, x_test, y_test, param_grid):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                               cv=3, n_jobs=-1, verbose=2)

    grid_search.fit(x_train, y_train)
    print(f'Parameter terbaik : {grid_search.best_params_} \n')
    best_grid = grid_search.best_estimator_
    best_grid.fit(x_train, y_train)
    y_pred = best_grid.predict(x_test)
    print(f'Akurasi : {accuracy_score(y_test, y_pred):,.4f}')
    print('---------- Classification Report ----------')
    print(f'{classification_report(y_test, y_pred)}\n')
    print('---------- Confusion Matrix ----------')
    conf = confusion_matrix(y_test, y_pred)
    print(conf)
    
    return best_grid


In [24]:
def cross_validation(model, x_train, y_train):
    Skfold = StratifiedKFold(n_splits=5)
    acc = cross_val_score(model, x_train, y_train, cv=Skfold)
    print(f'Hasil rata-rata akurasi : {acc.mean()}')
    print(f'Rata-rata std : {acc.std()}')
    print(
        f'Akurasi agar tidak overfit / underfit {(acc.mean() - acc.std()):,.4f} - {(acc.mean() + acc.std()):,.4f} \n')


In [195]:
latih(RandomForestClassifier(), x_train, y_train, x_test, y_test)


Akurasi : 0.9550 

---------- Classification Report ----------
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1084
           1       0.97      0.83      0.89       315

    accuracy                           0.95      1399
   macro avg       0.96      0.91      0.93      1399
weighted avg       0.96      0.95      0.95      1399


---------- Confusion Matrix ----------
[[1076    8]
 [  55  260]]


In [196]:
latih(RandomForestClassifier(), x_train_smote,
      y_train_smote, x_test_smote, y_test_smote)


Akurasi : 0.9806 

---------- Classification Report ----------
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1598
           1       1.00      0.97      0.98      1655

    accuracy                           0.98      3253
   macro avg       0.98      0.98      0.98      3253
weighted avg       0.98      0.98      0.98      3253


---------- Confusion Matrix ----------
[[1590    8]
 [  55 1600]]


In [197]:
best_model = latih_hyperparameter(RandomForestClassifier(), x_train,
                     y_train, x_test, y_test, param_grid)


Fitting 3 folds for each of 360 candidates, totalling 1080 fits
Parameter terbaik : {'bootstrap': True, 'max_depth': 110, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100} 

Akurasi : 0.9421
---------- Classification Report ----------
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1084
           1       0.99      0.75      0.85       315

    accuracy                           0.94      1399
   macro avg       0.96      0.87      0.91      1399
weighted avg       0.95      0.94      0.94      1399


---------- Confusion Matrix ----------
[[1081    3]
 [  78  237]]


In [198]:
best_model = latih_hyperparameter(RandomForestClassifier(), x_train_smote,
                     y_train_smote, x_test_smote, y_test_smote, param_grid)


Fitting 3 folds for each of 360 candidates, totalling 1080 fits
Parameter terbaik : {'bootstrap': True, 'max_depth': 100, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 200} 

Akurasi : 0.9671
---------- Classification Report ----------
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1598
           1       0.99      0.95      0.97      1655

    accuracy                           0.97      3253
   macro avg       0.97      0.97      0.97      3253
weighted avg       0.97      0.97      0.97      3253


---------- Confusion Matrix ----------
[[1579   19]
 [  88 1567]]


## Data Stemming

In [11]:
df_stemming = pd.read_csv('preprocess_fix_backslash.csv')
df_stemming.head()


,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK,aaceab,aacebdbd,ab,abad,abadi,abah,...,zuhur,zulkarnaen,zulkarnain,zulkifli,zuqfaiugqy,zvjqohw,zwcxnxhd,zwnbzkjn,zy,zynga
0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
x_stem = df_stemming.drop(columns=['PORNOGRAFI', 'SARA',
                                    'RADIKALISME', 'PENCEMARAN_NAMA_BAIK'])
x_stem.head()


,aaceab,aacebdbd,ab,abad,abadi,abah,abai,abang,abangan,abas,...,zuhur,zulkarnaen,zulkarnain,zulkifli,zuqfaiugqy,zvjqohw,zwcxnxhd,zwnbzkjn,zy,zynga
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
y_stem = df_stemming['PORNOGRAFI']
y_stem.head()


0    0
1    0
2    0
3    0
4    0
Name: PORNOGRAFI, dtype: int64

In [14]:
oversample = SMOTE()
x_stem_smote, y_stem_smote = oversample.fit_resample(x_stem, y_stem)

counter = Counter(y_stem_smote)
print(f'After : {counter}')


After : Counter({0: 5421, 1: 5421})


In [15]:
print(x_stem_smote.shape)
print(y_stem_smote.shape)


(10842, 19892)
(10842,)


In [16]:
x_stem_train, x_stem_test, y_stem_train, y_stem_test = train_test_split(
    x_stem, y_stem, stratify=y_stem, test_size=0.2, random_state=32)


In [17]:
x_stem_train_smote, x_stem_test_smote, y_stem_train_smote, y_stem_test_smote = train_test_split(
    x_stem_smote, y_stem_smote, stratify=y_stem_smote, test_size=0.2, random_state=32)


In [218]:
latih(RandomForestClassifier(), x_stem_train, y_stem_train, x_stem_test, y_stem_test)


Akurasi : 0.9528 

---------- Classification Report ----------
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1084
           1       0.98      0.81      0.89       315

    accuracy                           0.95      1399
   macro avg       0.96      0.90      0.93      1399
weighted avg       0.95      0.95      0.95      1399


---------- Confusion Matrix ----------
[[1078    6]
 [  60  255]]


In [219]:
latih(RandomForestClassifier(), x_stem_train_smote,
      y_stem_train_smote, x_stem_test_smote, y_stem_test_smote)


Akurasi : 0.9820 

---------- Classification Report ----------
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1084
           1       0.99      0.97      0.98      1085

    accuracy                           0.98      2169
   macro avg       0.98      0.98      0.98      2169
weighted avg       0.98      0.98      0.98      2169


---------- Confusion Matrix ----------
[[1075    9]
 [  30 1055]]


In [220]:
latih_hyperparameter(RandomForestClassifier(), x_stem_train,
                     y_stem_train, x_stem_test, y_stem_test, param_grid)


Fitting 3 folds for each of 360 candidates, totalling 1080 fits
Parameter terbaik : {'bootstrap': True, 'max_depth': 110, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 12, 'n_estimators': 100} 

Akurasi : 0.9435
---------- Classification Report ----------
              precision    recall  f1-score   support

           0       0.94      1.00      0.96      1084
           1       0.98      0.76      0.86       315

    accuracy                           0.94      1399
   macro avg       0.96      0.88      0.91      1399
weighted avg       0.95      0.94      0.94      1399


---------- Confusion Matrix ----------
[[1080    4]
 [  75  240]]


In [18]:
latih_hyperparameter(RandomForestClassifier(), x_stem_train_smote,
                     y_stem_train_smote, x_stem_test_smote, y_stem_test_smote, param_grid)


Fitting 3 folds for each of 360 candidates, totalling 1080 fits
Parameter terbaik : {'bootstrap': True, 'max_depth': 110, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 400} 

Akurasi : 0.9723
---------- Classification Report ----------
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1084
           1       0.99      0.95      0.97      1085

    accuracy                           0.97      2169
   macro avg       0.97      0.97      0.97      2169
weighted avg       0.97      0.97      0.97      2169


---------- Confusion Matrix ----------
[[1075    9]
 [  51 1034]]


## Modelling 4 Labels

In [ ]:
# model = RandomForestClassifier()

# for label in labels:
#     y = df[label]
#     X_train, X_test, y_train, y_test = train_test_split(
#         x, y, stratify=y, test_size=0.2, random_state=32)

#     model.fit(X_train, y_train)

#     y_pred = model.predict(X_test)

#     print(f'Akurasi untuk {label} : {accuracy_score(y_test, y_pred):,.2f}')

#     Skfold = StratifiedKFold(n_splits=5)
#     acc = cross_val_score(model, X_train, y_train, cv=Skfold)
#     print(f'Hasil rata-rata akurasi Cross Validation : {acc.mean()}')
#     print(f'Rata-rata std Cross Validation : {acc.std()}')


Akurasi untuk PORNOGRAFI : 0.95
Hasil rata-rata akurasi Cross Validation : 0.9472826503255458
Rata-rata std Cross Validation : 0.004905372194183675
Akurasi untuk SARA : 0.90
Hasil rata-rata akurasi Cross Validation : 0.8833109919571045
Rata-rata std Cross Validation : 0.006019581074792571
Akurasi untuk RADIKALISME : 0.94
Hasil rata-rata akurasi Cross Validation : 0.9240535235541938
Rata-rata std Cross Validation : 0.008984405713702948
Akurasi untuk PENCEMARAN_NAMA_BAIK : 0.81
Hasil rata-rata akurasi Cross Validation : 0.8080754500191498
Rata-rata std Cross Validation : 0.011847323725986092


# Modelling Pipeline

In [19]:
df_kata = pd.read_csv('data hasil/data_kalimat.csv')
df_kata.head()

,text,source,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK
0,woi anjing bodoh provinsi paling banyak ngerus...,kaskus,0,0,0,1
1,saja nitizen penasaran keluarga besar sudah pa...,instagram,0,0,0,0
2,sidangahok semoga sipenista agama ateknya mati...,twitter,0,1,1,1
3,jakarta barusan baca undang tetap dibedakan ko...,instagram,0,0,0,0
4,buat anak melulu kamu nof mikir apa kasian ana...,kaskus,0,0,0,0


In [42]:
df_kata.isnull().sum()

text                    0
source                  0
PORNOGRAFI              0
SARA                    0
RADIKALISME             0
PENCEMARAN_NAMA_BAIK    0
dtype: int64

In [48]:
df_kata = df_kata.dropna()

In [20]:
df_kata

,text,source,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK
0,woi anjing bodoh provinsi paling banyak ngerus...,kaskus,0,0,0,1
1,saja nitizen penasaran keluarga besar sudah pa...,instagram,0,0,0,0
2,sidangahok semoga sipenista agama ateknya mati...,twitter,0,1,1,1
3,jakarta barusan baca undang tetap dibedakan ko...,instagram,0,0,0,0
4,buat anak melulu kamu nof mikir apa kasian ana...,kaskus,0,0,0,0
...,...,...,...,...,...,...
6986,mas sebenarnya aku mau buly mas seperti nya ma...,instagram,0,0,0,0
6987,kdang mengaku nicky minaj kdang beyonce kdang ...,instagram,0,0,0,0
6988,waktu kemas tadi terfikir jugak botol air keci...,twitter,0,0,1,0
6989,heh kontol jan sok pemes kamu rp muka memek or...,twitter,1,0,0,1


In [25]:
df_kata.to_csv('data hasil/data_kalimat.csv', index=False)

NameError: name 'df_kata' is not defined

In [26]:
from sklearn.pipeline import Pipeline
import pickle


In [23]:
best_model = latih_hyperparameter(RandomForestClassifier(), x_train_smote,
                                  y_train_smote, x_test_smote, y_test_smote, param_grid)


Fitting 3 folds for each of 360 candidates, totalling 1080 fits
Parameter terbaik : {'bootstrap': True, 'max_depth': 100, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 200} 

Akurasi : 0.9640
---------- Classification Report ----------
              precision    recall  f1-score   support

           0       0.94      0.99      0.96      1060
           1       0.99      0.94      0.96      1109

    accuracy                           0.96      2169
   macro avg       0.97      0.96      0.96      2169
weighted avg       0.97      0.96      0.96      2169


---------- Confusion Matrix ----------
[[1053    7]
 [  71 1038]]


In [24]:
best_model

RandomForestClassifier(max_depth=100, min_samples_leaf=3, min_samples_split=10,
                       n_estimators=200)

In [55]:
removeFactory.get_stop_words()


['yang',
 'untuk',
 'pada',
 'ke',
 'para',
 'namun',
 'menurut',
 'antara',
 'dia',
 'dua',
 'ia',
 'seperti',
 'jika',
 'jika',
 'sehingga',
 'kembali',
 'dan',
 'tidak',
 'ini',
 'karena',
 'kepada',
 'oleh',
 'saat',
 'harus',
 'sementara',
 'setelah',
 'belum',
 'kami',
 'sekitar',
 'bagi',
 'serta',
 'di',
 'dari',
 'telah',
 'sebagai',
 'masih',
 'hal',
 'ketika',
 'adalah',
 'itu',
 'dalam',
 'bisa',
 'bahwa',
 'atau',
 'hanya',
 'kita',
 'dengan',
 'akan',
 'juga',
 'ada',
 'mereka',
 'sudah',
 'saya',
 'terhadap',
 'secara',
 'agar',
 'lain',
 'anda',
 'begitu',
 'mengapa',
 'kenapa',
 'yaitu',
 'yakni',
 'daripada',
 'itulah',
 'lagi',
 'maka',
 'tentang',
 'demi',
 'dimana',
 'kemana',
 'pula',
 'sambil',
 'sebelum',
 'sesudah',
 'supaya',
 'guna',
 'kah',
 'pun',
 'sampai',
 'sedangkan',
 'selagi',
 'sementara',
 'tetapi',
 'apakah',
 'kecuali',
 'sebab',
 'selain',
 'seolah',
 'seraya',
 'seterusnya',
 'tanpa',
 'agak',
 'boleh',
 'dapat',
 'dsb',
 'dst',
 'dll',
 'dahulu

In [34]:
tfidf = TfidfVectorizer(stop_words=removeFactory.get_stop_words(), analyzer='word')

pl = Pipeline([
    ('tf-idf', tfidf),
    ('model', best_model)
])


In [42]:
pl.fit(df_kata['text'], df_kata['PORNOGRAFI'])


Pipeline(steps=[('tf-idf',
                 TfidfVectorizer(stop_words=['yang', 'untuk', 'pada', 'ke',
                                             'para', 'namun', 'menurut',
                                             'antara', 'dia', 'dua', 'ia',
                                             'seperti', 'jika', 'jika',
                                             'sehingga', 'kembali', 'dan',
                                             'tidak', 'ini', 'karena', 'kepada',
                                             'oleh', 'saat', 'harus',
                                             'sementara', 'setelah', 'belum',
                                             'kami', 'sekitar', 'bagi', ...])),
                ('model',
                 RandomForestClassifier(max_depth=100, min_samples_leaf=3,
                                        min_samples_split=10,
                                        n_estimators=200))])

In [66]:
kalimat = 'hahaha'

In [67]:
pl.predict([kalimat])

array([0], dtype=int64)

In [68]:
pl.predict_proba([kalimat])

array([[0.95295063, 0.04704937]])

In [63]:
with open('model_pornografi', 'wb') as picklefile:
    pickle.dump(pl,picklefile)

In [ ]:
# def pipe(model, x, y, stop_words):
#     tfidf = TfidfVectorizer(
#         stop_words=stop_words, analyzer='word')

#     pl = Pipeline([
#         ('tf-idf', tfidf),
#         ('model', model)
#     ])

#     pl.fit(x, df[y])
#     with open(f'model {y}', 'wb') as picklefile:
#         pickle.dump(pl,picklefile)

In [54]:
df.head()

,PORNOGRAFI,SARA,RADIKALISME,PENCEMARAN_NAMA_BAIK,ab,abad,abadi,abadinya,abah,abai,...,zuhur,zulkarnaen,zulkarnain,zulkifli,zuqfaiugqy,zvjqohw,zwcxnxhd,zwnbzkjn,zy,zynga
0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Kebutuhan

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from nltk.tokenize import WordPunctTokenizer
import itertools
from pandas_profiling import ProfileReport
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
import pickle


labels = ['SARA', 'RADIKALISME', 'PENCEMARAN_NAMA_BAIK', 'PORNOGRAFI']


# factory = StopWordRemoverFactory()
# stopwords = factory.get_stop_words()

removeFactory = StopWordRemoverFactory()
stopwords = removeFactory.create_stop_word_remover()

vectorizer = TfidfVectorizer()

df = pd.read_csv('data hasil/preprocess_wo_stemming_fix_backslash.csv')
df_kata = pd.read_csv('data hasil/data_kalimat.csv')

x = df.drop(columns=['PORNOGRAFI', 'SARA',
            'RADIKALISME', 'PENCEMARAN_NAMA_BAIK'])

model = RandomForestClassifier()

param_grid = {
    'bootstrap': [True, False],
    'max_depth': [80, 90, 100, 110],
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 400, 500]
}


def latih(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(f'Akurasi : {accuracy_score(y_test, y_pred):,.4f} \n')
    print('---------- Classification Report ----------')
    print(f'{classification_report(y_test, y_pred)}\n')
    print('---------- Confusion Matrix ----------')
    conf = confusion_matrix(y_test, y_pred)
    print(conf)


def latih_hyperparameter(model, x_train, y_train, x_test, y_test, param_grid):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                               cv=3, n_jobs=-1, verbose=2)

    grid_search.fit(x_train, y_train)
    print(f'Parameter terbaik : {grid_search.best_params_} \n')
    best_grid = grid_search.best_estimator_
    best_grid.fit(x_train, y_train)
    y_pred = best_grid.predict(x_test)
    print(f'Akurasi : {accuracy_score(y_test, y_pred):,.4f}')
    print('---------- Classification Report ----------')
    print(f'{classification_report(y_test, y_pred)}\n')
    print('---------- Confusion Matrix ----------')
    conf = confusion_matrix(y_test, y_pred)
    print(conf)

    return best_grid


In [ ]:
# def pipeline_model(x, y, model, stopwords):
#     y = df[label]
#     x_smote, y_smote = oversample.fit_resample(x, y)

#     x_train_smote, x_test_smote, y_train_smote, y_test_smote = train_test_split(
#         x_smote, y_smote, test_size=0.2, random_state=42)

#     best_model = latih_hyperparameter(RandomForestClassifier(), x_train_smote,
#                                       y_train_smote, x_test_smote, y_test_smote, param_grid)

#     tfidf = TfidfVectorizer(
#         stop_words=removeFactory.get_stop_words(), analyzer='word')

#     pl = Pipeline([
#         ('tf-idf', tfidf),
#         ('model', best_model)
#     ])

#     pl.fit(df_kata['text'], df_kata[label])

#     pl = Pipeline([
#         ('tf-idf', tfidf),
#         ('model', best_model)
#     ])

#     with open(f'data hasil/{label}', 'wb') as picklefile:
#         pickle.dump(pl, picklefile)


In [3]:
for label in labels:
    print(f'========== {label} ==========')

    
    y = df[label]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    best_model = latih_hyperparameter(RandomForestClassifier(), x_train,
                                      y_train, x_test, y_test, param_grid)
    
    # x_smote, y_smote = oversample.fit_resample(x, y)
    
    # x_train_smote, x_test_smote, y_train_smote, y_test_smote = train_test_split(
    #     x_smote, y_smote, test_size=0.2, random_state=42)
    
    # best_model = latih_hyperparameter(RandomForestClassifier(), x_train_smote,
    #                                   y_train_smote, x_test_smote, y_test_smote, param_grid)
    
    tfidf = TfidfVectorizer(
        stop_words=removeFactory.get_stop_words(), analyzer='word')

    pl = Pipeline([
        ('tf-idf', tfidf),
        ('model', best_model)
    ])
    
    pl.fit(df_kata['text'], df_kata[label])

    pl = Pipeline([
        ('tf-idf', tfidf),
        ('model', best_model)
    ])
    
    with open(f'model/no smote {label}', 'wb') as picklefile:
        pickle.dump(pl, picklefile)


========== SARA ==========
Fitting 3 folds for each of 360 candidates, totalling 1080 fits
Parameter terbaik : {'bootstrap': True, 'max_depth': 110, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 100} 

Akurasi : 0.8563
---------- Classification Report ----------
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      1167
           1       0.79      0.18      0.29       232

    accuracy                           0.86      1399
   macro avg       0.83      0.59      0.61      1399
weighted avg       0.85      0.86      0.82      1399


---------- Confusion Matrix ----------
[[1156   11]
 [ 190   42]]


NameError: name 'Pipeline' is not defined

In [24]:
for label in labels:
    print(f'========== {label} ==========')

    
    y = df[label]
    x_smote, y_smote = oversample.fit_resample(x, y)
    
    x_train_smote, x_test_smote, y_train_smote, y_test_smote = train_test_split(
        x_smote, y_smote, test_size=0.2, random_state=42)
    
    best_model = latih_hyperparameter(RandomForestClassifier(), x_train_smote,
                                      y_train_smote, x_test_smote, y_test_smote, param_grid)
    
    tfidf = TfidfVectorizer(
        stop_words=removeFactory.get_stop_words(), analyzer='word')

    pl = Pipeline([
        ('tf-idf', tfidf),
        ('model', best_model)
    ])
    
    pl.fit(df_kata['text'], df_kata[label])

    pl = Pipeline([
        ('tf-idf', tfidf),
        ('model', best_model)
    ])
    
    with open(f'data hasil/{label}', 'wb') as picklefile:
        pickle.dump(pl, picklefile)


========== SARA ==========
Fitting 3 folds for each of 360 candidates, totalling 1080 fits
Parameter terbaik : {'bootstrap': True, 'max_depth': 90, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 12, 'n_estimators': 400} 

Akurasi : 0.9408
---------- Classification Report ----------
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      1189
           1       0.92      0.96      0.94      1160

    accuracy                           0.94      2349
   macro avg       0.94      0.94      0.94      2349
weighted avg       0.94      0.94      0.94      2349


---------- Confusion Matrix ----------
[[1098   91]
 [  48 1112]]
========== RADIKALISME ==========
Fitting 3 folds for each of 360 candidates, totalling 1080 fits
Parameter terbaik : {'bootstrap': True, 'max_depth': 90, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 200} 

Akurasi : 0.9499
---------- Classification Report --